<a href="https://colab.research.google.com/github/Amarnath-188/Design-Optimization/blob/Project-1/Thermodynamic_parameter_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Introduction**



The Wilson activity coefficient model is a semi-empirical model used to describe the non-ideal behavior of liquid mixtures. It is based on the assumption that the excess Gibbs free energy of mixing is due to the different sizes and interactions of the molecules in the mixture. The Wilson model has been shown to be accurate for a wide range of mixtures, including ethanol-cyclohexane.

# **Ebulliometer**


An ebulliometer is a laboratory instrument used to measure the boiling point of a liquid. It works by heating a liquid until it boils, and then measuring the temperature of the vapor. The boiling point of a liquid is related to its vapor pressure, which is a measure of the tendency of the liquid to evaporate.

#**Vapor Liquid Equilibrium (VLE)**

VLE is a state of equilibrium in which a liquid and its vapor are present at the same temperature and pressure. The composition of the liquid and vapor phases is determined by the following relationship:

